### **Environment Setup & State Reset**

In [1]:
#To print the full path to the Python interpreter that is currently running your script or notebook.
import sys
print(sys.executable)


/Users/rohitkhanna/memosynth--lite/venv/bin/python


In [3]:
#To add the parent directory of your current script or notebook to Python’s module search path.
import sys, os
sys.path.append(os.path.abspath(".."))


In [26]:
#Import your APIs and sample memories
from memosynth.vector_store import write_memory
from memosynth.timeline_store import log_memory
from memosynth.memory_client import (query_memory, summarize_memories, diff, resolve, example_memory)
import duckdb
import pandas as pd

In [5]:
#To delete any existing collection of memories to avoid overlap or confusion
from qdrant_client import QdrantClient

client = QdrantClient("http://localhost:6333")
client.delete_collection(collection_name="memos")


True

In [19]:
# Clear the timeline table to avoid duplicate entries in a demo
con.execute("DELETE FROM memory_log")


### **Define Memories**

In [21]:
#Define three memories
memory1 = example_memory
memory2 = {
    "id": "m-002",
    "project": "demo_project",
    "agent": "doc_bot",
    "summary": "Client is expecting an up-to-date forecast of Q3 to plan ahead for pitfalls.",
    "type": "insight",
    "tags": ["finance", "Q3", "forecast","planning","pitfalls"],
    "source": "Finance_Forecast_Q3.pdf",
    "author": "doc_bot",
    "created_at": "2025-06-22",
    "version": 1,
    "confidence": 0.8,
    "visibility": "project",
    "sensitivity": "medium"
}
memory3 = {
    "id": "m-003",
    "project": "demo_project",
    "agent": "doc_bot",
    "summary": "Client expressed concern about rising costs in Q2.",
    "type": "insight",
    "tags": ["finance", "Q2", "costs"],
    "source": "Finance_Report_Q2.pdf",
    "author": "doc_bot",
    "created_at": "2025-06-22",
    "version": 1,
    "confidence": 0.92,
    "visibility": "project",
    "sensitivity": "medium"
}

### **Store Memories (Vector, Timeline, Graph)**

In [23]:
#Write and log the memories
write_memory(memory1)
log_memory(memory1)
write_memory(memory2)
log_memory(memory2)
write_memory(memory3)
log_memory(memory3)

In [36]:
# Connect to the DuckDB timeline database
con = duckdb.connect('memory_timeline.db')

# Query all timeline entries ordered by timestamp descending
query = "SELECT * FROM memory_log ORDER BY timestamp DESC"
  


In [53]:
from memosynth.graph_store import create_memory_node, relate_memories, find_related_memories

# Create nodes if not already present
create_memory_node(memory1)
create_memory_node(memory2)
create_memory_node(memory3)


### **Graph Memory**

In [15]:
#Establish relation between the memory nodes based on your analysis
relate_memories(memory1["id"], memory2["id"], rel_type="RELATED_TO")
relate_memories(memory3["id"], memory1["id"], rel_type="FOLLOWS")
relate_memories(memory3["id"], memory2["id"], rel_type="CAUSES")

In [55]:
# Find all memories related to "m-001" within 2 hops in the memory graph,
# then print each related memory's ID and summary.
related = find_related_memories("m-001", hops=2)
for rid, summary in related:
    print(f"{rid}: {summary}")


m-003: Client expressed concern about rising costs in Q2.
m-002: Client is expecting an up-to-date forecast of Q3 to plan ahead for pitfalls.


### **Visualize Timeline**

In [41]:
# Fetch the timeline as a pandas DataFrame
df_timeline = con.execute(query).fetchdf()

# Display the DataFrame as a table in the notebook
display(df_timeline)

,id,summary,timestamp,version
0,m-002,Client is expecting an up-to-date forecast of ...,2025-06-22,1
1,m-003,Client expressed concern about rising costs in...,2025-06-22,1
2,m-001,Client asked about margin drop in Q2.,2025-06-19,1


### **Query Memories (Semantic Search)**

In [21]:
#Query for Q2 issues
results = query_memory("Q2 issues")
print("Query Results for 'Projections':")
for mem in results:
    print(mem["summary"])

Query Results for 'Projections':
Client expressed concern about rising costs in Q2.
Client asked about margin drop in Q2.
Client is expecting an up-to-date forecast of Q3 to plan ahead for pitfalls.


### **Summarize Memories (Using LLM)**

In [23]:
#Summarize all memories
all_memories = [memory1, memory2, memory3]
summary = summarize_memories(all_memories)
print("\nSummary of all memories:")
print(summary)


Summary of all memories:
The client has noted a decline in their profits due to margin drops, and they are looking for an up-to-date forecast of Q3 to plan ahead for potential pitfalls. The client is concerned about rising costs in Q2 due to fluctuating market conditions.


### **Compare and Resolve**

In [46]:
#Show diff between two memories
print("\nDiff between memory1 and memory2:")
print(diff(memory1, memory2))



Diff between memory1 and memory2:
⚠️ Difference in summaries:
1: Client asked about margin drop in Q2.
2: Client is expecting an up-to-date forecast of Q3 to plan ahead for pitfalls.


In [49]:
#Resolve contradiction between two memories
print("\nResolution between memory1 and memory2:")
print(resolve(memory1, memory2))


Resolution between memory1 and memory2:
Resolved: The client's concern about a margin drop in Q2 was resolved by the fact that they had previously received a detailed forecast of Q3, which will help them prepare for any potential pitfalls.
